# 1. Import Libraries

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sb
import plotly.graph_objects as go

import geojson
import country_converter as coco # to convert and match country names

In [2]:
df = pd.read_csv('data/merged.csv')

## DensityMap

In [3]:
# Display the number of prizes won by the top 10 countries
df['bornCountry'].value_counts().head(10)

USA                293
United Kingdom      92
Germany             67
France              56
Sweden              30
Japan               28
Canada              21
the Netherlands     19
Switzerland         19
Italy               18
Name: bornCountry, dtype: int64

In [4]:
df_country_points = pd.read_csv('data/country_points.csv')
df_country = df_country_points.rename(columns={'country' : 'iso-a2'})

In [5]:
with open('data/countries.geojson') as f:
    data_geo = geojson.load(f)

# In order to feed the GeoJson into Plotly,
# Adding the 'id' key to each feature containing the value of each countries ISO-A3 code 
for feature in data_geo['features']:
    feature['id'] = feature['properties']['ISO_A3'] 

In [6]:
# Grouping the dataset by country and counting 
df_by_country = df[['id', 'bornCountryCode']].groupby('bornCountryCode').count().rename(columns={"id": "count"})
df_by_country

,count
bornCountryCode,
AR,4
AT,19
AU,10
AZ,1
BA,2
...,...
VE,1
VN,1
YE,1


In [7]:
df_density = pd.merge(df_country, df_by_country, left_on='iso-a2', right_on='bornCountryCode')
df_density

,iso-a2,latitude,longitude,name,count
0,AR,-38.416097,-63.616672,Argentina,4
1,AT,47.516231,14.550072,Austria,19
2,AU,-25.274398,133.775136,Australia,10
3,AZ,40.143105,47.576927,Azerbaijan,1
4,BA,43.915886,17.679076,Bosnia and Herzegovina,2
...,...,...,...,...,...
77,VE,6.423750,-66.589730,Venezuela,1
78,VN,14.058324,108.277199,Vietnam,1
79,YE,15.552727,48.516388,Yemen,1
80,ZA,-30.559482,22.937506,South Africa,9


In [8]:
df_density.sort_values(by=['count'])

,iso-a2,latitude,longitude,name,count
40,IS,64.963051,-19.020835,Iceland,1
30,GR,39.074208,21.824312,Greece,1
32,HR,45.100000,15.200000,Croatia,1
34,ID,-0.789275,113.921327,Indonesia,1
38,IQ,33.223191,43.679291,Iraq,1
...,...,...,...,...,...
63,PL,51.919438,19.145136,Poland,31
26,FR,46.227638,2.213749,France,59
19,DE,51.165691,10.451526,Germany,84
27,GB,55.378051,-3.435973,United Kingdom,108


In [9]:
data_densitymap = dict(type='densitymapbox', 
                       lat=df_density['latitude'], 
                       lon=df_density['longitude'],
                       z=np.log(df_density['count']),
                       text=df_density['name'],
                       colorscale='inferno',
                       colorbar=dict(title='Total Number of Nobel Prizes (log scaled)'),
                       hovertemplate='Country: %{text} <br>' + 'Number of Nobel Prizes: %{z}'
                      )


layout_densitymap = dict(mapbox=dict(style='white-bg',
                                     layers=[dict(source=feature, below='traces',type='fill',
                                                  fill=dict(outlinecolor='gray')) 
                                             for feature in data_geo['features']]
                                      ),
                         title=dict(text='World Nobel Prizes Density Map',
                                    x=.5 # Title relative position according to the xaxis, range (0,1)
                                   )
                           )

In [10]:
fig_densitymap = go.Figure(data=data_densitymap, 
                           layout=layout_densitymap)

fig_densitymap.show(renderer='browser')

## ScatterMapBox

In [11]:
df_density.head()

,iso-a2,latitude,longitude,name,count
0,AR,-38.416097,-63.616672,Argentina,4
1,AT,47.516231,14.550072,Austria,19
2,AU,-25.274398,133.775136,Australia,10
3,AZ,40.143105,47.576927,Azerbaijan,1
4,BA,43.915886,17.679076,Bosnia and Herzegovina,2


In [12]:
cc = coco.CountryConverter()
df_density['iso-a3'] = cc.convert(df_density['iso-a2'], to='ISO3')
df_density.head()

,iso-a2,latitude,longitude,name,count,iso-a3
0,AR,-38.416097,-63.616672,Argentina,4,ARG
1,AT,47.516231,14.550072,Austria,19,AUT
2,AU,-25.274398,133.775136,Australia,10,AUS
3,AZ,40.143105,47.576927,Azerbaijan,1,AZE
4,BA,43.915886,17.679076,Bosnia and Herzegovina,2,BIH


In [13]:
data_choroplethmap = dict(type='choroplethmapbox', 
                          geojson=data_geo,
                          locations=df_density['iso-a3'],
                          z=np.log(df_density['count']),
                          colorscale='inferno',
                          colorbar=dict(title='Total Number of Nobel Prizes'),
                         )


layout_choroplethmap = dict(mapbox=dict(style='white-bg',
                            layers=[dict(source=feature,
                                         below='traces',
                                         type='fill',
                                         fill=dict(outlinecolor='gray')
                                         ) for feature in data_geo['features']]
                                        ),
                            title=dict(text='Total Number of Nobel Prizes - Choropleth Map',
                                       x=.5 # Title relative position according to the xaxis, range (0,1)
                                       )
                           )

In [14]:
fig_choroplethmap = go.Figure(data=data_choroplethmap, 
                              layout=layout_choroplethmap)

fig_choroplethmap.show(renderer='browser')